In [ ]:
!pip install pandas scikit-learn streamlit python-docx PyPDF2 pdfplumber --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 71.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 73.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 71.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 57.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import re
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# For file handling
import os
from PyPDF2 import PdfReader
import docx


In [ ]:
import zipfile
import os
zip_path = "/content/Resume_Dataset.zip"
extract_path = "/content/Resume_Dataset/extracted/"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# List files to check
os.listdir(extract_path)


['Resume.md', 'Resume.csv']

In [ ]:
import pandas as pd

csv_file = os.path.join(extract_path, "Resume.csv")

df = pd.read_csv(csv_file, encoding='utf-8-sig', quotechar='"')
print(df.columns)


Index(['resume_id,category,resume_text,skills_list,experience_years'], dtype='object')


In [ ]:
# Remove BOM, quotes, spaces
df.columns = [c.strip().replace('"','').replace("ï»¿","") for c in df.columns]

# Rename resume_text for easier access
df.rename(columns={'resume_text':'resume_clean'}, inplace=True)

print(df.columns)


Index(['resume_id,category,resume_text,skills_list,experience_years'], dtype='object')


In [ ]:
import pandas as pd

df = pd.read_csv('/content/Resume_Dataset/extracted/Resume.csv', encoding='utf-8-sig')
df.head()


,"resume_id,category,resume_text,skills_list,experience_years"
0,"1,HR,hr administrator marketing associate hr a..."
1,"2,HR,hr specialist us hr operations summary ve..."
2,"3,HR,hr director summary over years experience..."
3,"4,HR,hr specialist summary dedicated driven an..."
4,"5,HR,hr manager skill highlights hr skills hr ..."


In [ ]:
import json

with open("/content/skills_vocab.json", "r") as f:
    skills_vocab = set(json.load(f))

print("Skills loaded:", len(skills_vocab))


Skills loaded: 197


In [ ]:
import pandas as pd

csv_file = "/content/Resume_Dataset/extracted/Resume.csv"

# Load as single column
df = pd.read_csv(csv_file, header=None, encoding='utf-8-sig')

# The first row is header
header = df.iloc[0,0].replace('"','').split(',')

# Remaining rows are data
df = df[1:]
df = df[0].str.split(',', expand=True)
df.columns = header

# Optional: reset index
df = df.reset_index(drop=True)

# Check columns
print(df.columns)


Index(['resume_id', 'category', 'resume_text', 'skills_list',
       'experience_years'],
      dtype='object')


In [ ]:
import re

# Clean text function
def clean_text(t):
    if pd.isna(t): return ""
    t = str(t)
    t = t.encode("ascii","ignore").decode()
    t = re.sub(r'http\S+|www\S+', ' ', t)
    t = re.sub(r'[\r\n\t]+', ' ', t)
    t = re.sub(r'[^a-zA-Z0-9 .,]', ' ', t)
    t = re.sub(r'\s+', ' ', t)
    return t.strip().lower()

# Extract skills based on vocab
def extract_skills(text, vocab):
    text = clean_text(text)
    return [skill for skill in vocab if skill.lower() in text.lower()]

df['skills_extracted'] = df['resume_text'].apply(lambda x: extract_skills(x, skills_vocab))

# Quick check
df[['resume_text','skills_extracted']].head()


,resume_text,skills_extracted
0,hr administrator marketing associate hr admini...,"[documentation, aws, operations, ece, support,..."
1,hr specialist us hr operations summary versati...,"[communication, operations, ece, recruitment, ..."
2,hr director summary over years experience in r...,"[classification, compliance, ece, recruitment,..."
3,hr specialist summary dedicated driven and dyn...,"[communication, documentation, ece, presentati..."
4,hr manager skill highlights hr skills hr depar...,"[aws, operations, compliance, teamwork, recrui..."


In [ ]:
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# Encode categories
le = LabelEncoder()
df['category_encoded'] = le.fit_transform(df['category'])

# Multi-hot encode skills
mlb = MultiLabelBinarizer()
skills_matrix = mlb.fit_transform(df['skills_extracted'])
skills_df = pd.DataFrame(skills_matrix, columns=mlb.classes_)

# Combine features
df_final = pd.concat([df[["resume_text","category_encoded"]].reset_index(drop=True),
                      skills_df.reset_index(drop=True)], axis=1)

X = df_final.drop(columns=["resume_text","category_encoded"])
y = df_final["category_encoded"]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Random Forest
clf = RandomForestClassifier(n_estimators=200, random_state=42)
clf.fit(X_train, y_train)

print(" Model trained successfully")


 Model trained successfully


In [ ]:
from PyPDF2 import PdfReader
import docx

# Read PDF
def read_pdf(file_path):
    text = ""
    reader = PdfReader(file_path)
    for page in reader.pages:
        t = page.extract_text()
        if t: text += t + " "
    return text

# Read DOCX
def read_docx(file_path):
    doc = docx.Document(file_path)
    text = " ".join([p.text for p in doc.paragraphs])
    return text


In [ ]:
!pip install streamlit pandas fuzzywuzzy python-Levenshtein pyngrok pdfplumber python-docx nltk --quiet
!python -m nltk.downloader punkt


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 86.3 MB/s eta 0:00:00
<frozen runpy>:128: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
import pdfplumber
from docx import Document
import re
import nltk
from nltk.tokenize import word_tokenize

# PDF text extraction
def read_pdf(file):
    text = ""
    with pdfplumber.open(file) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + " "
    return text

# DOCX text extraction
def read_docx(file):
    doc = Document(file)
    text = " ".join([p.text for p in doc.paragraphs])
    return text

# Basic text cleaning
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^a-zA-Z0-9 ,]', '', text)
    return text

# Skill extraction
def extract_skills(text, skills_vocab):
    tokens = word_tokenize(text)
    skills = [token for token in tokens if token in skills_vocab]
    return list(set(skills))


In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
from utils import read_pdf, read_docx, clean_text, extract_skills

st.title("AI Resume Screening Dashboard")

# Sample skills vocabulary
skills_vocab = ["python","excel","sql","machine learning","nlp","communication"]

uploaded_file = st.file_uploader("Upload your Resume (PDF or DOCX)", type=["pdf","docx"])

if uploaded_file:
    # Extract text
    if uploaded_file.name.endswith(".pdf"):
        resume_text = read_pdf(uploaded_file)
    elif uploaded_file.name.endswith(".docx"):
        resume_text = read_docx(uploaded_file)

    resume_clean = clean_text(resume_text)
    skills = extract_skills(resume_clean, skills_vocab)

    st.subheader("Detected Skills:")
    st.write(skills)

    st.subheader("Resume Text Preview:")
    st.write(resume_clean[:500] + "...")


Writing app.py


In [ ]:
# Step 1: Install required packages
!pip install python-docx PyPDF2 nltk fuzzywuzzy python-Levenshtein --quiet

import nltk
nltk.download('punkt')

import json
import pandas as pd
import re
from fuzzywuzzy import fuzz
from docx import Document
import PyPDF2
from google.colab import files

# Step 2: Upload skills_vocab.json
print("Upload your skills_vocab.json")
uploaded = files.upload()
skills_vocab_file = list(uploaded.keys())[0]

with open(skills_vocab_file, 'r') as f:
    skills_vocab = json.load(f)

# Step 3: Helper functions
def read_pdf(file):
    pdf_reader = PyPDF2.PdfReader(file)
    text = ""
    for page in pdf_reader.pages:
        text += page.extract_text()
    return text

def read_docx(file):
    doc = Document(file)
    return "\n".join([para.text for para in doc.paragraphs])

def clean_text(text):
    text = text.lower()
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'[^a-zA-Z0-9,.\s]', '', text)
    return text

def extract_skills(text, skills_vocab):
    detected = []
    for skill in skills_vocab:
        if fuzz.partial_ratio(skill.lower(), text) > 80:
            detected.append(skill)
    return detected

# Step 4: Upload your resume
print("Upload your resume (PDF or DOCX)")
uploaded_resume = files.upload()
resume_file = list(uploaded_resume.keys())[0]

if resume_file.endswith(".pdf"):
    resume_text = read_pdf(resume_file)
elif resume_file.endswith(".docx"):
    resume_text = read_docx(resume_file)
else:
    raise ValueError("File must be PDF or DOCX")

# Step 5: Clean and extract skills
resume_clean = clean_text(resume_text)
skills_found = extract_skills(resume_clean, skills_vocab)

# Step 6: Show results
print("\n--- Resume Text Preview ---\n")
print(resume_text[:1000])  # Show first 1000 characters
print("\n--- Detected Skills ---\n")
print(skills_found)


Upload your skills_vocab.json


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Saving skills_vocab.json to skills_vocab (1).json
Upload your resume (PDF or DOCX)


Saving Swathi_NagaDevi_ML_Resume.pdf to Swathi_NagaDevi_ML_Resume (1).pdf

--- Resume Text Preview ---

 S W A T H I   N A G A   D E V I
 Ø=ÜÞ   + 9 1 - 7 5 6 9 7 9 8 9 3 6   |  Ø=Üç   d o l l y y 8 4 8 @ g m a i l . c o m
Ø=Ý   L i n k e d I n :   l i n k e d i n . c o m / i n / n a g a d e v i - p e r a b a t h u l a   |   G i t H u b :   g i t h u b . c o m / S w a t h i 8 8 - d e v i
 M L   &   A I   O P E R A T I O N S   A S S O C I A T E
 P r o f e s s i o n a l   S u m m a r y
 A n a l y t i c a l   a n d   s e l f - d r i v e n   E C E   g r a d u a t e   s k i l l e d   i n   P y t h o n ,   T e n s o r F l o w ,   a n d   M L   m o d e l   b u i l d i n g .
 D e l i v e r e d   h i g h - a c c u r a c y   p r o j e c t s   i n   i m a g e   c l a s s i f i c a t i o n   a n d   d a t a   s e g m e n t a t i o n   u s i n g   r e a l - w o r l d
 d a t a s e t s .   E a g e r   t o   a p p l y   t e c h n i c a l   r i g o r   a n d   d a t a - d r i v e n   t h i n k i n g  

/usr/local/lib/python3.12/dist-packages/PyPDF2/_cmap.py:142: PdfReadWarning: Advanced encoding /UniJIS-UCS2-H not implemented yet
  warnings.warn(


In [ ]:
# Interactive Resume Fit Checker
import json, re
import pdfplumber
import nltk
from nltk.tokenize import word_tokenize
from google.colab import files
from pdfminer.pdfdocument import PDFSyntaxError

nltk.download('punkt')
nltk.download('punkt_tab')


# -------------------------------
# Load skills vocabulary
# -------------------------------
with open('skills_vocab.json') as f:
    skills_vocab = json.load(f)

# -------------------------------
# Functions
# -------------------------------
def read_pdf(file_path):
    text = ''
    try:
        with pdfplumber.open(file_path) as pdf:
            for page in pdf.pages:
                text += page.extract_text() + ' '
    except PDFSyntaxError:
        print(f"Error: Could not read PDF file '{file_path}'. It might be corrupted or not a valid PDF.")
        return None
    except Exception as e:
        print(f"An unexpected error occurred while reading PDF file '{file_path}': {e}")
        return None
    return text

def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def extract_skills(text, skills_vocab):
    words = set(word_tokenize(text))
    return [skill for skill in skills_vocab if skill.lower() in words]

# -------------------------------
# Job Description Input
# -------------------------------
jd_text = input("Paste Job Description here: ")
jd_clean = clean_text(jd_text)
jd_skills = extract_skills(jd_clean, skills_vocab)
print("\nJob Skills Required:", jd_skills)

# -------------------------------
# Upload Resume PDF(s)
# -------------------------------
uploaded = files.upload()  # Drag & drop PDF(s)

# -------------------------------
#  Process Resumes
# -------------------------------
for filename in uploaded:
    resume_text = read_pdf(filename)
    if resume_text is not None:
        resume_clean = clean_text(resume_text)
        resume_skills = extract_skills(resume_clean, skills_vocab)

        matched_skills = set(resume_skills).intersection(set(jd_skills))
        fit_score = len(matched_skills) / len(jd_skills) * 100 if jd_skills else 0

        print("\nResume:", filename)
        print("Detected Skills:", resume_skills)
        print("Matched Skills:", list(matched_skills))
        print("Missing Skills:", list(set(jd_skills) - set(resume_skills)))
        print("Resume Fit %: {:.2f}%".format(fit_score))
    else:
        print(f"Skipping analysis for file '{filename}' due to reading error.")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Paste Job Description here: We are looking for a Machine Learning Engineer (Fresher) to join our AI/ML team. You will be responsible for assisting in building and deploying machine learning models for various data-driven applications.  Responsibilities:  Data collection, cleaning, and preprocessing  Feature engineering and exploratory data analysis  Implementing ML algorithms using Python (scikit-learn, pandas, numpy)  Supporting model evaluation, validation, and optimization  Collaborating with data engineers and analysts to improve pipelines  Required Skills:  Strong programming skills in Python  Experience with pandas, numpy, matplotlib, seaborn  Knowledge of machine learning algorithms and basic statistics  Familiarity with SQL and data visualization tools  Understanding of natural language processing or computer vision (bonus)  Excellent problem-solving and communication skills  Education:  B.Tech / M.Tech / MSc in Computer Science, Data Science, or related field.

Job Skills Requ

Saving Swathi_NagaDevi_ML_Resume.pdf to Swathi_NagaDevi_ML_Resume (2).pdf

Resume: Swathi_NagaDevi_ML_Resume (2).pdf
Detected Skills: ['classification', 'regression', 'tensorflow', 'keras', 'excel', 'annotation', 'operations', 'communication', 'ece', 'microcontroller', 'arduino', 'verilog', 'pcb', 'python', 'c', 'r', 'pandas', 'numpy', 'matplotlib', 'seaborn']
Matched Skills: ['python', 'pandas', 'numpy', 'seaborn', 'communication', 'matplotlib']
Missing Skills: ['sql', 'statistics']
Resume Fit %: 75.00%


In [3]:
!pip install streamlit pdfplumber python-docx nltk


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 68.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 50.7 MB/s eta 0:00:00


In [5]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
!pip install pdfplumber

In [7]:
!pip install python-docx

In [8]:
!pip install pdfplumber python-docx gradio nltk


In [9]:
import pdfplumber
from docx import Document
print("All libraries loaded successfully!")


All libraries loaded successfully!


In [17]:
# =========================================
# AI Resume Fit Checker — Enhanced ATS + Dashboard
# =========================================

!pip install pdfplumber python-docx gradio nltk sentence-transformers plotly -q

import pdfplumber
from docx import Document
import gradio as gr
import nltk
from nltk.corpus import stopwords
import re
from sentence_transformers import SentenceTransformer, util
import torch
import plotly.graph_objects as go

# ----------------------------
# NLTK Stopwords
# ----------------------------
nltk.download('stopwords', quiet=True)
stop_words = set(stopwords.words('english'))

# ----------------------------
# Semantic Skills Dictionary
# ----------------------------
SKILL_SYNONYMS = {
    "ml": ["machine learning", "ml"],
    "ai": ["artificial intelligence", "ai"],
    "python": ["python"],
    "sql": ["sql"],
    "tensorflow": ["tensorflow"],
    "keras": ["keras"],
    "react": ["react", "reactjs"],
    "java": ["java"],
    "pandas": ["pandas"],
    "numpy": ["numpy"]
}

# ----------------------------
# Education Synonyms
# ----------------------------
EDU_SYNONYMS = {
    "btech": ["b.tech", "btech", "bachelor of technology", "bachelor in technology"],
    "mtech": ["m.tech", "mtech", "master of technology"],
    "bsc": ["b.sc", "bsc", "bachelor of science"],
    "msc": ["m.sc", "msc", "master of science"],
    "mba": ["mba", "master of business administration"]
}

CERTIFICATIONS = ["aws","gcp","azure","kaggle","coursera"]
HEADINGS = ["experience","education","skills","projects","certifications"]

# ----------------------------
# Load embedding model
# ----------------------------
model = SentenceTransformer('all-MiniLM-L6-v2')

# ----------------------------
# Extract text from resume
# ----------------------------
def extract_text_from_file(file):
    text = ""
    if file.name.endswith(".pdf"):
        with pdfplumber.open(file.name) as pdf:
            for page in pdf.pages:
                text += page.extract_text() or ""
    elif file.name.endswith(".docx"):
        doc = Document(file.name)
        for para in doc.paragraphs:
            text += para.text + "\n"
    else:
        raise ValueError("Unsupported file type. Please upload PDF or DOCX.")
    return text.strip()

# ----------------------------
# Preprocess text
# ----------------------------
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r"[^a-z0-9\s]", " ", text)
    tokens = [word for word in text.split() if word not in stop_words and len(word) > 2]
    return tokens

# ----------------------------
# Section Extraction
# ----------------------------
def extract_sections(text):
    sections = {}
    for heading in HEADINGS:
        pattern = rf'{heading}.*?(?=\n[A-Z][a-z]+:|\Z)'
        match = re.search(pattern, text, re.DOTALL|re.IGNORECASE)
        if match:
            sections[heading] = match.group(0)
    return sections

# ----------------------------
# Semantic Skills Score
# ----------------------------
def get_skills_score_semantic(resume_text, jd_text, similarity_threshold=0.7):
    resume_text_lower = resume_text.lower()
    jd_text_lower = jd_text.lower()

    matched_skills = set()
    missing_skills = set()

    # Known skills matching
    for skill, synonyms in SKILL_SYNONYMS.items():
        jd_has_skill = any(s in jd_text_lower for s in synonyms)
        resume_has_skill = any(s in resume_text_lower for s in synonyms)
        if jd_has_skill and resume_has_skill:
            matched_skills.add(skill)
        elif jd_has_skill and not resume_has_skill:
            missing_skills.add(skill)

    # Semantic matching for phrases
    resume_phrases = [line.strip() for line in resume_text.split('\n') if line.strip()]
    jd_phrases = [line.strip() for line in jd_text.split('\n') if line.strip()]

    if resume_phrases and jd_phrases:
        resume_embeddings = model.encode(resume_phrases, convert_to_tensor=True)
        jd_embeddings = model.encode(jd_phrases, convert_to_tensor=True)
        cos_sim_matrix = util.cos_sim(resume_embeddings, jd_embeddings)

        for i, r_phrase in enumerate(resume_phrases):
            for j, jd_phrase in enumerate(jd_phrases):
                if cos_sim_matrix[i][j] >= similarity_threshold:
                    matched_skills.add(r_phrase)

    score = len(matched_skills) / (len(matched_skills) + len(missing_skills)) if (matched_skills | missing_skills) else 0
    return round(score,2), matched_skills, missing_skills

# ----------------------------
# Experience Score
# ----------------------------
def ats_experience_score(resume_text, jd_text):
    years = re.findall(r'(\d+)\s+years?', resume_text.lower())
    resume_years = max([int(y) for y in years]) if years else 0

    jd_years = re.findall(r'(\d+)\+?\s+years?', jd_text.lower())
    jd_required = int(jd_years[0]) if jd_years else 0

    score = min(resume_years/jd_required,1) if jd_required else 0
    return round(score,2)

# ----------------------------
# Education Score
# ----------------------------
def ats_education_score(resume_text):
    resume_text_lower = resume_text.lower()

    degree_score = 1 if any(d in resume_text_lower for d in sum(EDU_SYNONYMS.values(),[])) else 0
    cert_score = 1 if any(c in resume_text_lower for c in CERTIFICATIONS) else 0
    return round(0.7*degree_score + 0.3*cert_score,2)

# ----------------------------
# Achievements Score
# ----------------------------
def ats_achievement_score(resume_text):
    metrics = re.findall(r'\d+[%]?', resume_text)
    score = min(len(metrics)/5,1)
    return round(score,2)

# ----------------------------
# Formatting Score
# ----------------------------
def ats_format_score(resume_text):
    heading_count = sum(1 for h in HEADINGS if h in resume_text.lower())
    heading_score = heading_count / len(HEADINGS)

    bullets = len(re.findall(r'•|\*|-', resume_text))
    bullet_score = 1 if bullets >= 3 else 0.5
    return round(0.7*heading_score + 0.3*bullet_score,2)

# ----------------------------
# Dynamic ATS Weights
# ----------------------------
def get_dynamic_weights(jd_text):
    jd_lines = jd_text.lower().split("\n")
    skills_lines = sum(1 for line in jd_lines if any(s in line for s in sum(SKILL_SYNONYMS.values(),[])))
    experience_lines = sum(1 for line in jd_lines if "year" in line)
    education_lines = sum(1 for line in jd_lines if any(d in line for d in sum(EDU_SYNONYMS.values(),[])))
    achievement_lines = sum(1 for line in jd_lines if re.search(r'\d+[%]?', line))

    total = skills_lines + experience_lines + education_lines + achievement_lines
    if total == 0: total = 1
    weights = {
        "skills": skills_lines/total,
        "experience": experience_lines/total,
        "education": education_lines/total,
        "achievement": achievement_lines/total,
        "formatting": 0.1
    }
    sum_w = sum(weights.values())
    for k in weights: weights[k] /= sum_w
    return weights

# ----------------------------
# Total ATS Score
# ----------------------------
def ats_total_score(resume_text,jd_text,skills_score):
    experience = ats_experience_score(resume_text,jd_text)
    education = ats_education_score(resume_text)
    achievement = ats_achievement_score(resume_text)
    formatting = ats_format_score(resume_text)

    weights = get_dynamic_weights(jd_text)

    total = round(
        weights["skills"]*skills_score +
        weights["experience"]*experience +
        weights["education"]*education +
        weights["achievement"]*achievement +
        weights["formatting"]*formatting, 2
    )
    breakdown = {"skills":skills_score,"experience":experience,"education":education,"achievement":achievement,"formatting":formatting}
    return total, breakdown

# ----------------------------
# Recommendations
# ----------------------------
def generate_recommendations(missing_skills, breakdown):
    recs = []
    if missing_skills:
        recs.append(f"Consider adding or highlighting these skills: {', '.join(missing_skills)}")
    if breakdown['experience'] < 0.5:
        recs.append("Highlight relevant experience more clearly.")
    if breakdown['formatting'] < 0.7:
        recs.append("Improve resume formatting (headings, bullets, consistency).")
    if breakdown['education'] < 0.7:
        recs.append("Include relevant degrees or certifications.")
    if breakdown['achievement'] < 0.5:
        recs.append("Add measurable achievements (metrics, KPIs).")
    return "\n".join(recs) if recs else "No major recommendations. Resume looks strong!"

# ----------------------------
# Dashboard (kept as-is)
# ----------------------------
def create_dashboard(breakdown):
    categories = list(breakdown.keys())
    values = [v*100 for v in breakdown.values()]

    fig = go.Figure([go.Bar(x=categories, y=values, marker_color='teal')])
    fig.update_layout(
        title="Resume ATS Sub-Scores",
        yaxis=dict(title="Score (%)", range=[0, 100]),
        xaxis=dict(title="Factors"),
        template="plotly_white"
    )
    return fig

# ----------------------------
# Project / Job Title Detection
# ----------------------------
PROJECT_KEYWORDS = ["github","kaggle","portfolio","project"]
JOB_TITLE_KEYWORDS = ["engineer","analyst","developer","manager","intern"]

def detect_projects_and_titles(resume_text):
    resume_lower = resume_text.lower()
    projects = [p for p in PROJECT_KEYWORDS if p in resume_lower]
    job_titles = [j for j in JOB_TITLE_KEYWORDS if j in resume_lower]
    return projects, job_titles

# ----------------------------
# Main Analysis
# ----------------------------
def analyze_resume_and_jd(resume_file, resume_text, jd_text):
    try:
        if resume_file is not None:
            resume_content = extract_text_from_file(resume_file)
        elif resume_text:
            resume_content = resume_text
        else:
            return " Please upload or paste your resume text.", None, None, None, None, None, None, None, None

        if not jd_text:
            return "Please enter a Job Description (JD).", None, None, None, None, None, None, None, None

        # Skills
        skills_score, matched_skills, missing_skills = get_skills_score_semantic(resume_content, jd_text)
        # ATS Total with dynamic weights
        total_score, breakdown = ats_total_score(resume_content,jd_text,skills_score)
        # Dashboard
        dashboard_fig = create_dashboard(breakdown)
        # Optional project / title detection
        projects, job_titles = detect_projects_and_titles(resume_content)
        # Recommendations
        recommendations = generate_recommendations(missing_skills, breakdown)

        return (
            resume_content[:2000]+"..." if len(resume_content) > 2000 else resume_content,
            f"ATS Total Score: {total_score*100}%",
            f"Skills Score: {skills_score*100}%, Matched: {', '.join(matched_skills)}",
            f"Missing Skills: {', '.join(missing_skills)}",
            f"Experience / Education / Achievements / Formatting: {breakdown['experience']*100}%, {breakdown['education']*100}%, {breakdown['achievement']*100}%, {breakdown['formatting']*100}%",
            dashboard_fig,
            f"Projects Detected: {', '.join(projects) if projects else 'None'}",
            f"Job Titles Detected: {', '.join(job_titles) if job_titles else 'None'}",
            recommendations
        )
    except Exception as e:
        return f"Error: {str(e)}", None, None, None, None, None, None, None, None

# ----------------------------
# Gradio Interface
# ----------------------------
with gr.Blocks() as demo:
    gr.Markdown("## Resume Fit Checker — Enhanced ATS + Dashboard")

    with gr.Row():
        with gr.Column():
            resume_file = gr.File(label=" Upload Resume (PDF/DOCX)")
            resume_text = gr.Textbox(lines=5, placeholder="Or paste resume text here...", label="Or paste resume text")
            jd_text = gr.Textbox(lines=5, placeholder="Paste Job Description (JD) here...", label="Job Description (JD)")
            analyze_button = gr.Button("Analyze Resume & JD")

        with gr.Column():
            resume_preview = gr.Textbox(label="Resume Preview (extracted)")
            match_score = gr.Textbox(label="ATS Total Score")
            matched_skills_box = gr.Textbox(label="Matched Skills")
            missing_skills_box = gr.Textbox(label="Missing Skills")
            other_scores = gr.Textbox(label="Experience / Education / Achievements / Formatting")
            dashboard = gr.Plot(label="ATS Sub-Score Dashboard")
            projects_box = gr.Textbox(label="Projects Detected")
            job_titles_box = gr.Textbox(label="Job Titles Detected")
            recommendations_box = gr.Textbox(label="Recommendations")

    analyze_button.click(
        analyze_resume_and_jd,
        inputs=[resume_file, resume_text, jd_text],
        outputs=[resume_preview, match_score, matched_skills_box, missing_skills_box, other_scores, dashboard, projects_box, job_titles_box, recommendations_box]
    )

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a6dcca9fe7afe5dce0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
